In [1]:
import numpy as np
seed = 0
np.random.seed(0)


def eggholder(x):
    return (-(x[1] + 47) * np.sin(np.sqrt(abs(x[0]/2 + (x[1]  + 47)))) - \
            x[0] * np.sin(np.sqrt(abs(x[0] - (x[1]  + 47)))))

bounds = [(-512, 512), (-512, 512)]

In [2]:
import time

def eggholder_objective(x):
    '''Evaluates x on the eggholder function (minimizing)
    
    x: a list/array of length 2 with each value in [0, 1]
    '''
    start = time.time()
    bound_range = [bounds[0][1] - bounds[0][0]]
    bound_range.append(bounds[1][1] - bounds[1][0])
    bound_min = [bounds[0][0]]
    bound_min.append(bounds[1][0])
    x_new = x.copy()
    for i in range(len(x)):
        x_new[i] = bound_min[i] + x[i] * bound_range[i]
    
    # the fitness which determines the selection process in DE
    ## the lower the fitness value, the better is x adjudged to be
    fitness = eggholder(x_new)
    # runtime of objective, or None, or equivalent
    cost = time.time() - start 
    
    return fitness, cost

In [5]:
%%time

### introducing sleep

sleep_low = 0.2
sleep_high = 0.5

def eggholder_objective_sleep(x):
    '''Evaluates x on the eggholder function (minimizing)
    
    x: a list/array of length 2 with each value in [0, 1]
    '''
    start = time.time()
    bound_range = [bounds[0][1] - bounds[0][0]]
    bound_range.append(bounds[1][1] - bounds[1][0])
    bound_min = [bounds[0][0]]
    bound_min.append(bounds[1][0])
    x_new = x.copy()
    for i in range(len(x)):
        x_new[i] = bound_min[i] + x[i] * bound_range[i]
    
    # the fitness which determines the selection process in DE
    ## the lower the fitness value, the better is x adjudged to be
    fitness = eggholder(x_new)
    # 25% chance of a long sleep of 2 seconds to simulate bottlenecks
    if np.random.uniform() > 0.75:
        time.sleep(2)
    else:
        time.sleep(np.random.uniform(low=sleep_low, high=sleep_high))
    # runtime of objective, or None, or equivalent
    cost = time.time() - start 
    
    return fitness, cost

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 36.7 µs


In [11]:
from denas import DE, PDE, AsyncPDE

import dask

In [12]:
# Sequential

de = DE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=20,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
)
start = time.time()
traj, runtime, hist = de.run(generations=10, verbose=True)
end = time.time() - start
    
print("Time taken for sequential processing: {} seconds".format(end))
print(de.inc_config, de.inc_score)

Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /10 -- -547.8635
Generation 2 /10 -- -547.8635
Generation 3 /10 -- -794.7629
Generation 4 /10 -- -794.7629
Generation 5 /10 -- -794.7629
Generation 6 /10 -- -794.7629
Generation 7 /10 -- -794.7629
Generation 8 /10 -- -794.7629
Generation 9 /10 -- -794.7629
Generation 10/10 -- -794.7629

Run complete!
Time taken for sequential processing: 164.73409366607666 seconds
[0.06276866 0.88686323] -794.7628545499281


In [13]:
# Parallel (1 worker)

num_workers = 1

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=20,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=10, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /10 -- -700.0901
Generation 2 /10 -- -700.0901
Generation 3 /10 -- -700.0901
Generation 4 /10 -- -858.958
Generation 5 /10 -- -858.958
Generation 6 /10 -- -858.958
Generation 7 /10 -- -858.958
Generation 8 /10 -- -858.958
Generation 9 /10 -- -858.958
Generation 10/10 -- -858.958

Run complete!
Time to initialise client: 1.6244685649871826 seconds
Time taken for parallel processing with 1 workers: 171.242333650589 seconds
[0.92344193 0.94715412] -858.9580073022762


In [14]:
# Parallel (2 workers)

num_workers = 2

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=20,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=10, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38479 instead
  http_address["port"], self.http_server.port
/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36111 instead
  http_address["port"], self.http_server.port


Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /10 -- -695.8255
Generation 2 /10 -- -695.8255
Generation 3 /10 -- -695.8255
Generation 4 /10 -- -695.8255
Generation 5 /10 -- -695.8255
Generation 6 /10 -- -695.8255
Generation 7 /10 -- -695.8255
Generation 8 /10 -- -695.8255
Generation 9 /10 -- -695.8255
Generation 10/10 -- -888.9939

Run complete!
Time to initialise client: 0.7789802551269531 seconds
Time taken for parallel processing with 2 workers: 83.06812119483948 seconds
[0.05053659 0.87313113] -888.9939168254796


In [20]:
# Parallel (4 workers)

num_workers = 4

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=20,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=10, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36507 instead
  http_address["port"], self.http_server.port
/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33265 instead
  http_address["port"], self.http_server.port


Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /10 -- -640.3775
Generation 2 /10 -- -640.3775
Generation 3 /10 -- -640.3775
Generation 4 /10 -- -640.3775
Generation 5 /10 -- -640.3775
Generation 6 /10 -- -640.3775
Generation 7 /10 -- -640.3775
Generation 8 /10 -- -640.3775
Generation 9 /10 -- -640.3775
Generation 10/10 -- -640.3775

Run complete!
Time to initialise client: 2.8334078788757324 seconds
Time taken for parallel processing with 4 workers: 53.59642672538757 seconds
[0.78116439 0.00561923] -640.3774870297616


In [16]:
# Parallel (20 workers)

num_workers = 20

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=20,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=10, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40293 instead
  http_address["port"], self.http_server.port
/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37235 instead
  http_address["port"], self.http_server.port


Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /10 -- -365.7444
Generation 2 /10 -- -622.2672
Generation 3 /10 -- -734.3517
Generation 4 /10 -- -734.3517
Generation 5 /10 -- -734.3517
Generation 6 /10 -- -734.3517
Generation 7 /10 -- -734.3517
Generation 8 /10 -- -734.3517
Generation 9 /10 -- -734.3517
Generation 10/10 -- -881.0744

Run complete!
Time to initialise client: 4.030801296234131 seconds
Time taken for parallel processing with 20 workers: 29.560590028762817 seconds
[0.0426116  0.87121828] -881.0743968267418


In [17]:
import dask
dask.config.set(scheduler='processes', num_workers=1)

In [18]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client.cluster

/media/neeratyoy/Mars/Freiburg/Hiwi/DE/envs/dask/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34927 instead
  http_address["port"], self.http_server.port


In [19]:
client

Client Scheduler: tcp://127.0.0.1:33271 Dashboard: http://127.0.0.1:34927/status,Cluster Workers: 4 Cores: 8 Memory: 8.09 GB
